In [1]:
%load_ext autoreload
%autoreload 2

# Import library

In [1]:
import numpy as np
import math
# from . import calc_util as cu
import sys
sys.path.append('src')
import enex_analysis as enex
import dartwork_mpl as dm
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np
from tqdm import tqdm
import pandas as pd


Load colors...
Load colormaps...


# 1. 단일 운전점 최적화 & 사이클 도식

In [2]:
# =============================================================================
# 1) 단일 운전점 최적화 & 사이클 도식
# =============================================================================
GSHPB = enex.GroundSourceHeatPumpBoiler2(
    refrigerant  = 'R410A',
    disp_cmp     = 0.00005, #0.00001 ~ 0.00011m³/rev
    eta_cmp_isen = 0.7,
    eta_cmp_dV   = 0.85,
    T_f_bh_in    = 15.0,
    Tg           = 15.0,
    UA_HX_tank       = 500,  # W/K
    UA_HX_water_loop = 500,  # W/K
    D_b = 0,                 # Borehole depth [m]
    H_b = 200,               # Borehole height [m]
    r_b = 0.08,              # Borehole radius [m]
    R_b = 0.108,             # Effective borehole thermal resistance [mK/W]
    dV_f = 24,               # Volumetric flow rate of fluid [L/min]
    k_g   = 2.0,
    c_g   = 800,
    rho_g = 2000,
    E_pmp = 200,
)

ref_loop_result = GSHPB._find_ref_loop_optimal_operation(T_w_tank=60, Q_load=-6000)

print(
    f'T1: {enex.K2C(ref_loop_result["T1"]):.2f} °C,\n'
    f'T2: {enex.K2C(ref_loop_result["T2"]):.2f} °C,\n'
    f'T3: {enex.K2C(ref_loop_result["T3"]):.2f} °C,\n'
    f'T4: {enex.K2C(ref_loop_result["T4"]):.2f} °C,\n'
    f'E_cmp: {ref_loop_result["E_cmp"]:.1f} W,\n'
    f'cmp_rps: {ref_loop_result["cmp_rps"]:.2f} rps,\n'
    f'cmp_rpm: {ref_loop_result["cmp_rps"]*60:.0f} rpm,\n'
)

GSHPB.plot_cycle_diagrams(result=ref_loop_result)

/home/habin/Codes/En_system_Ex_analysis/src/enex_analysis/HeatPump_model.py:1117: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  axi.set_ylim(*ylims[idx])
findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'cursive' not found because none of the following families were found: Apple Chancery, Textile, Zapf Chancery, Sand, Script MT, Felipa, Comic Neue, Comic Sans MS, cursive


T1: 8.09 °C,
T2: 95.26 °C,
T3: 62.25 °C,
T4: 8.03 °C,
E_cmp: 1956.6 W,
cmp_rps: 22.06 rps,
cmp_rpm: 1324 rpm,



# 2. 장기 시뮬레이션 설정

In [25]:
# =============================================================================
# 2) 장기 시뮬레이션 설정
# =============================================================================
GSHPB = enex.GroundSourceHeatPumpBoiler2(
    refrigerant  = 'R410A',
    disp_cmp     = 0.00005,
    eta_cmp_isen = 0.7,
    eta_cmp_dV   = 0.85,
    T_f_bh_in    = 15.0,
    Tg           = 15.0,
    UA_HX_tank       = 500,  # W/K
    UA_HX_water_loop = 500,  # W/K
    D_b = 0,                 # Borehole depth [m]
    H_b = 200,               # Borehole height [m]
    r_b = 0.08,              # Borehole radius [m]
    R_b = 0.108,             # Effective borehole thermal resistance [mK/W]
    dV_f = 24,               # Volumetric flow rate of fluid [L/min]
    k_g   = 2.0,
    c_g   = 800,
    rho_g = 2000,
    E_pmp = 200,
)

# 시뮬레이션 타임축
simulation_period_sec = 10 * enex.d2h * enex.h2s  # 10일 [s]

# np.logspace는 0 포함 불가 → 1초부터 시작############################
# start_time = 1
# end_time = simulation_period_sec * enex.h2s
# time = np.logspace(np.log10(start_time), np.log10(end_time), num=tN)
# tN = 500
#######################################################################

# linear time step ######################################################
# time = np.linspace(0, simulation_period_sec*enex.h2s, num=tN)
dt = 1 * enex.m2s  # 1시간 간격
start_time = 0
end_time = simulation_period_sec
time = np.arange(start_time, end_time, dt)
tN = len(time)
print(tN)
#########################################################################

# Load profile 설정 (W) #####################################
''' 
Q_load_day1: 하루 24시간 급탕 부하 시간대 6~8 h, 18~20 h에 각각 -10000 W 부하 발생
Q_load_day2: 하루 24시간 급탕 부하 시간대 10~14 h에 각각 -10000 W 부하 발생
'''

Q_serv_schedule_day1 = [0, 0, 0, 0, 0, 0, # 0 h ~ 6 h
                        1, 1, 0, 0, 0, 0, # 7 h ~ 12 h
                        0, 0, 0, 0, 0, 1, # 13 h ~ 18 h
                        1, 0, 0, 0, 0, 0,]  # 19 h ~ 24 h

# Q_loss는 default로 1을 곱한 상태

# 현재 문제는 schedule이 0인 순간에 Q_load가 매우 작아 히트펌프 가동 조건 안되고, T_w_tank이 점점 떨어져야하지만, 현재 온도를 고정값으로 설정함.
#################################################################
# Q_load_2 = Q_load_day2 * (simulation_period_sec)


# 하중 및 탱크 목표온도

# 결과 버퍼
nan_arr = [np.nan] * tN
results = {
    # 온도 및 열량
    'T_bh': nan_arr.copy(), 'T_f_bh': nan_arr.copy(),
    'T_f_bh_in': nan_arr.copy(), 'T_f_bh_out': nan_arr.copy(),
    'Q_bh': nan_arr.copy(),

    # 각 컴포넌트별 엑서지 (입력, 출력, 파괴량)
    'Xin_g': nan_arr.copy(), 'Xout_g': nan_arr.copy(), 'Xc_g': nan_arr.copy(),
    'Xin_GHE': nan_arr.copy(), 'Xout_GHE': nan_arr.copy(), 'Xc_GHE': nan_arr.copy(),
    'Xin_HX': nan_arr.copy(), 'Xout_HX': nan_arr.copy(), 'Xc_HX': nan_arr.copy(),
    'Xin_r': nan_arr.copy(),  'Xout_r': nan_arr.copy(),  'Xc_r': nan_arr.copy(),
    'Xin_tank': nan_arr.copy(), 'Xout_tank': nan_arr.copy(), 'Xc_tank': nan_arr.copy(),
    'Xin_mix': nan_arr.copy(), 'Xout_mix': nan_arr.copy(), 'Xc_mix': nan_arr.copy(),
    'E_cmp': nan_arr.copy(), 'E_pmp': nan_arr.copy(), 'g_i': nan_arr.copy(),
    # 냉매 엑서지
    'x1': nan_arr.copy(), 'x2': nan_arr.copy(), 'x3': nan_arr.copy(), 'x4': nan_arr.copy(),

    # 시스템 효율
    'X_eff': nan_arr.copy(),
    'is_on': nan_arr.copy(),
    'g': nan_arr.copy(),
}

T_w_serv  = 45
T_w_sup   = 15
dV_w_serv = 8 * enex.L2m3/ enex.m2s
T0        = 20                     

# 초기 상태/상수
Q_bh_purse = np.zeros(tN)
Q_bh_old = 0

T0_K = enex.C2K(T0)                                    # 기준 온도 [K]
P0   = 101325                                          # 기준 압력 [kPa]
h0   = CP.PropsSI('H', 'P', P0, 'T', T0_K, GSHPB.ref)  # 기준 엔탈피 [J/kg]
s0   = CP.PropsSI('S', 'P', P0, 'T', T0_K, GSHPB.ref)  # 기준 엔트로피 [J/kgK]


# =============================================================================
# 2-1) 제어 로직 및 탱크 동특성 파라미터 설정
# =============================================================================
# ... (기존 GSHPB 객체 생성, time 배열 생성 등) ...

# --- (A) 탱크 동특성 파라미터 ---
r0 = 0.2; H = 0.8
UA_tank = enex.calc_simple_tank_UA(r0=r0, H=H) # W/K
V_tank = math.pi * r0**2 * H
C_tank = enex.c_w * enex.rho_w * V_tank # J/K, 탱크의 열용량

# --- (B) 온도 기반 제어(Thermostat) 파라미터 ---
T_w_tank_setpoint = 60.0  # 목표 상한 온도 (°C)
T_w_tank_lower_bound = 50.0 # 하한 온도 (°C)

# --- (C) 탱크 온도 상태배열 생성 (°C) ---
T_w_tank_init = 55.0  # 초기 탱크 온도 (상한과 하한 사이에서 시작)
T_w_tank = np.empty(tN)
T_w_tank[0] = T_w_tank_init

# --- (D) 급탕 사용 스케줄 생성 ---
serv_sched_day = np.array(Q_serv_schedule_day1, dtype=float)
serv_sched = serv_sched_day[np.mod(np.arange(tN), 24)]     

# --- (C) 탱크 온도 상태배열 생성 (°C) ---
T_w_tank_init = 55.0  # 기존에 scalar로 있던 초기 탱크온도(예: 60)
T_w_tank = np.empty(tN)
T_w_tank[0] = T_w_tank_init

# --- (D) (선택) 부하를 루프에서 일관되게 계산할 것이므로, 기존 Q_load는 참조만 하거나 건너뜀 ---
# Q_load는 컨트롤/비교용으로 남겨도 되지만, 루프 내부에서 q_load_n을 다시 계산해 사용합니다.

14400


# 3. 시간 스텝 루프

In [26]:

# ... (기존 results 딕셔너리, Q_bh_purse 등 초기화) ...

# =============================================================================
# 3) 시간 스텝 루프
# =============================================================================
for n in tqdm(range(tN), desc="Sim", leave=False):
    
    # --- (0-1) 현재 스텝의 손실량 계산 ---
    Q_env_loss = UA_tank * (T_w_tank[n] - T0)
    
    # alp 계산 및 급탕 사용 손실 계산
    den = max(1e-6, enex.C2K(T_w_tank[n]) - enex.C2K(T_w_sup))
    alp_n = min(1.0, max(0.0, (enex.C2K(T_w_serv) - enex.C2K(T_w_sup)) / den))
    dV_w_sup_tank = alp_n * dV_w_serv
    Q_use_loss = serv_sched[n] * (enex.c_w * enex.rho_w * dV_w_sup_tank * (T_w_tank[n] - T_w_sup))
    
    total_loss = Q_env_loss + Q_use_loss

    # --- (0-2) 탱크 온도 기반 제어 로직 ★★★ ---
    # 현재 히트펌프가 켜져 있는지 여부를 이전 스텝 상태로부터 추정 (n=0일 때는 꺼져있다고 가정)
    is_on_prev_step = results['is_on'][n-1] if n > 0 else False
    
    # 켤지 말지 결정
    if T_w_tank[n] < T_w_tank_lower_bound:
        is_on_current_step = True # 하한 온도 아래로 떨어지면 무조건 켠다.
    elif T_w_tank[n] > T_w_tank_setpoint:
        is_on_current_step = False # 상한 온도를 넘으면 무조건 끈다.
    else: # 데드밴드 안에서는 이전 상태를 유지한다.
        is_on_current_step = is_on_prev_step

    # 현재 스텝에서 히트펌프가 감당해야 할 부하(q_load_n) 결정
    if is_on_current_step:
        q_load_n = -5000
    else:
        # 히트펌프가 꺼져 있다면, 부하는 0이다.
        q_load_n = 0.0

    
    # --- (1) 최적 운전점 계산 ---
    # q_load_n을 바탕으로 최적 운전점을 찾는다. (q_load_n이 0이면 _off_result가 반환됨)
    ref_loop_result = GSHPB._find_ref_loop_optimal_operation(
        T_w_tank=T_w_tank[n],
        Q_load=q_load_n,
    )

    # ... (ref_loop_result is None 처리) ...
    results['is_on'][n] = ref_loop_result['is_on'] # is_on 상태 저장
    
    # --- (2) 탱크 온도 업데이트 (lumped capacitance) ---
    Q_tank_in = -ref_loop_result['Q_ref_tank'] # 히트펌프가 실제 공급한 열량
    Q_net = Q_tank_in - total_loss
    
    # if n % 1 == 0:
    #     print(f"Step {n}: T_w_tank = {T_w_tank[n]:.1f} °C, is_on = {is_on_current_step}, Q_net = {Q_net:.1f} W, total_loss = {total_loss:.1f} W")
    
    # 냉매 엑서지 (각 상태점) [J/kg]
    results['x1'][n] = (ref_loop_result['h1'] - h0) - (ref_loop_result['s1'] - s0) * T0_K
    results['x2'][n] = (ref_loop_result['h2'] - h0) - (ref_loop_result['s2'] - s0) * T0_K
    results['x3'][n] = (ref_loop_result['h3'] - h0) - (ref_loop_result['s3'] - s0) * T0_K
    results['x4'][n] = (ref_loop_result['h4'] - h0) - (ref_loop_result['s4'] - s0) * T0_K


    # --- (2) 보어홀 선형열속 Q_bh (W/m) ---
    Q_bh = (ref_loop_result['Q_ref_HX'] - GSHPB.E_pmp) / GSHPB.H_b

    # 펄스(증분) 기록
    if Q_bh != Q_bh_old:
        Q_bh_purse[n] = Q_bh - Q_bh_old
        Q_bh_old = Q_bh


    # 경과시간 컨볼루션 (펄스별 tau)
    pulses_idx = np.flatnonzero(Q_bh_purse) # dQ != 0 인 지점의 인덱스 array
    dQ = Q_bh_purse[pulses_idx] # 각 펄스 크기 (W/m)
    tau = np.maximum(time[n] - time[pulses_idx], 0.0) # 각 펄스 이후 경과시간
    g_n = np.array([enex.G_FLS(t=t, ks=GSHPB.k_g, as_=GSHPB.alp_g, rb=GSHPB.r_b, H=GSHPB.H_b) for t in tau])
    results['g'][n] = np.sum(g_n) # convolution 된 g 값 기록
    
    dT_bh = np.dot(dQ, g_n)
    T_bh = GSHPB.Tg - dT_bh

    # 유체 온도 (보어홀-유체 열저항 반영)
    T_f_bh = T_bh - Q_bh * GSHPB.R_b
    T_f_bh_in  = T_f_bh - Q_bh * GSHPB.H_b / (2 * enex.c_w * enex.rho_w * GSHPB.dV_f)
    T_f_bh_out = T_f_bh + Q_bh * GSHPB.H_b / (2 * enex.c_w * enex.rho_w * GSHPB.dV_f)

    # --- (3) 엑서지 계산 ---
    E_cmp = ref_loop_result['E_cmp']
    E_pmp = GSHPB.E_pmp
    Q_ref_tank = ref_loop_result['Q_ref_tank']  # 음수
    T3_K = ref_loop_result['T3']
    Q_ref_HX = ref_loop_result['Q_ref_HX']      # 양수
    T1_K = ref_loop_result['T1']

    # 일(Work)
    X_pmp = E_pmp
    X_cmp = E_cmp
    
    # 열(Heat)
    X_ref_int = Q_ref_tank * (1 - T0_K / T3_K)   # 냉매 -> 저탕조
    X_ref_HX  = Q_ref_HX  * (1 - T0_K / T1_K)   # GHE -> 냉매

    # 유체(Fluid)
    X_f_in  = enex.c_w * enex.rho_w * GSHPB.dV_f * ((T_f_bh_in  - T0_K) - T0_K * math.log(T_f_bh_in  / T0_K))
    X_f_out = enex.c_w * enex.rho_w * GSHPB.dV_f * ((T_f_bh_out - T0_K) - T0_K * math.log(T_f_bh_out / T0_K))

    # 지중 경계
    Q_g_total = Q_bh * GSHPB.H_b
    X_c = (1 - T0_K / GSHPB.Tg) * Q_g_total  # 초기 지중온도 경계
    X_b = (1 - T0_K / T_bh) * Q_g_total      # 보어홀 벽면온도 경계

    # 컴포넌트별 엑서지 입/출/파괴
    Xin_g,  Xout_g  = X_c, X_b
    Xc_g            = Xin_g  - Xout_g

    Xin_GHE  = X_pmp + Xout_g + X_f_in
    Xout_GHE = X_f_out
    Xc_GHE   = Xin_GHE - Xout_GHE

    Xin_HX  = Xout_GHE
    Xout_HX = X_ref_HX + X_f_in
    Xc_HX   = Xin_HX - Xout_HX

    Xin_r, Xout_r   = X_cmp + X_ref_HX, X_ref_int
    Xc_r            = Xin_r - Xout_r

    Xin_tank         = abs(X_ref_int)
    Xout_useful_tank = abs(q_load_n) * (1 - T0_K / enex.C2K(T_w_tank[n]))
    Xc_tank          = Xin_tank - Xout_useful_tank

    # 시스템 엑서지 효율
    X_in_total = X_pmp + X_cmp
    X_eff = Xout_useful_tank / X_in_total if X_in_total > 0 else 0

    # 결과 기록
    results['T_bh'][n]        = T_bh
    results['T_f_bh'][n]      = T_f_bh
    results['T_f_bh_in'][n]   = T_f_bh_in
    results['T_f_bh_out'][n]  = T_f_bh_out
    results['Q_bh'][n]        = Q_bh
    
    results['E_cmp'][n]      = E_cmp
    results['E_pmp'][n]      = E_pmp
    
    results['g_i']

    results['Xin_g'][n]       = Xin_g
    results['Xout_g'][n]      = Xout_g
    results['Xc_g'][n]        = Xc_g

    results['Xin_GHE'][n]     = Xin_GHE
    results['Xout_GHE'][n]    = Xout_GHE
    results['Xc_GHE'][n]      = Xc_GHE

    results['Xin_HX'][n]      = Xin_HX
    results['Xout_HX'][n]     = Xout_HX
    results['Xc_HX'][n]       = Xc_HX

    results['Xin_r'][n]       = Xin_r
    results['Xout_r'][n]      = Xout_r
    results['Xc_r'][n]        = Xc_r

    results['Xin_tank'][n]    = Xin_tank
    results['Xout_tank'][n]   = Xout_useful_tank
    results['Xc_tank'][n]     = Xc_tank

    results['Xc_mix'][n]      = np.nan

    results['X_eff'][n]       = X_eff

    # 순환 유체 입력온도 업데이트 (°C 입력)
    GSHPB.T_f_bh_in = T_f_bh_in
    if n < tN - 1:
        T_w_tank[n+1] = T_w_tank[n] + (Q_net / C_tank) * dt

# 4. CSV 저장

In [54]:
# =============================================================================
# 4) CSV 저장
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv('GSHPB_exergy_simulation_results.csv', index=False)

# 5. Plot

## 5.1 Data import

In [49]:
df = pd.read_csv('GSHPB_exergy_simulation_results.csv')
df['Xc_g']
np.min(df['T_bh'])
plt.plot(time, df['E_cmp'])

KeyError: 'E_cmp'

In [ ]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정
# =============================================================================
plt.figure(figsize=(dm.cm2in(12), dm.cm2in(7)))
# 온도 데이터를 섭씨(°C)로 변환하여 플로팅
results_df